<a href="https://colab.research.google.com/github/fred-lafleche/Exchange/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imports
from sklearn import preprocessing
from sklearn import tree
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plot
from sklearn.utils import resample
from scipy.sparse import coo_matrix
from sklearn.utils import resample
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
shoppersInput_file= "/content/drive/MyDrive/Colab Notebooks/online_shoppers_intention.csv"
heartInput_file = "/content/drive/MyDrive/Colab Notebooks/heart.csv"
marketingInput_file = "/content/drive/MyDrive/Colab Notebooks/marketing_campaign.csv"

shopDataPd = pd.read_csv(shoppersInput_file)
heartDataPd = pd.read_csv(heartInput_file)
marketDataPd = pd.read_csv(marketingInput_file)
print("Shopper Data: \n",shopDataPd.head())
print("\nHeart Data: \n",heartDataPd.head())
print("\nMarket Campaing Data: \n",marketDataPd.head())


Shopper Data: 
    Administrative  Administrative_Duration  ...  Weekend  Revenue
0               0                      0.0  ...    False    False
1               0                      0.0  ...    False    False
2               0                      0.0  ...    False    False
3               0                      0.0  ...    False    False
4               0                      0.0  ...     True    False

[5 rows x 18 columns]

Heart Data: 
    age  sex  cp  trestbps  chol  fbs  ...  exang  oldpeak  slope  ca  thal  target
0   63    1   3       145   233    1  ...      0      2.3      0   0     1       1
1   37    1   2       130   250    0  ...      0      3.5      0   0     2       1
2   41    0   1       130   204    0  ...      0      1.4      2   0     2       1
3   56    1   1       120   236    0  ...      0      0.8      2   0     2       1
4   57    0   0       120   354    0  ...      1      0.6      2   0     2       1

[5 rows x 14 columns]

Market Campaing Data: 
     

Online Shoppers Intention

In [5]:
# Normalization
dataArray = shopDataPd.values
np.random.shuffle(dataArray)
numericData = dataArray[:,[0,1,2,3,4,5,6,7,8,9]]
scaler = preprocessing.StandardScaler().fit(numericData)
scalledNumericData = scaler.transform(numericData)

# One hot encoding
categoricalData = dataArray[:,[10,15,16]]
categoricalLabel = dataArray[:,[17]]
encoder1 = preprocessing.OneHotEncoder()
encoder2 = preprocessing.OneHotEncoder()
encoder1.fit(categoricalData)
encoder2.fit(categoricalLabel)
categoricalData=encoder1.transform(categoricalData).toarray()
categoricalLabel = encoder2.transform(categoricalLabel).toarray()

#join data back together
shopData = np.column_stack((scalledNumericData, categoricalData))
#add back columns 11 to 14 and 17
shopData = np.column_stack((shopData, dataArray[:,[11,12,13,14]]))
shopLabel = categoricalLabel[:,[1]]

Heart Data

In [6]:
heartDataArray = heartDataPd.values
np.random.shuffle(heartDataArray)
print(heartDataArray.size)
heartData = np.delete(heartDataArray, 13, 1)
print(heartData.size)
heartLabel = heartDataArray[:,[13]]
print(heartLabel.size)

4242
3939
303


Marketing Campaign

In [7]:
marketDataArray = marketDataPd.values
np.random.shuffle(marketDataArray)
print(marketDataArray.size)
marketLabel = marketDataArray[:,[6]] #label
marketData = np.delete(marketDataArray, 6, 1) #remove label
marketData = np.delete(marketData, 6, 1) #date
marketData = np.delete(marketData, 3, 1) #Marital Status
marketData = np.delete(marketData, 2, 1) #Education

# Normalization
scalerMarket = preprocessing.StandardScaler().fit(marketData)
scalledMarketData = scalerMarket.transform(marketData)

# One hot encoding
categoricalMarketData = marketDataArray[:,[2,3]]
encoder = preprocessing.OneHotEncoder()
encoder.fit(categoricalMarketData)
categoricalMarketData=encoder.transform(categoricalMarketData).toarray()

encoder = preprocessing.OneHotEncoder()
encoder.fit(marketLabel)
marketLabel=encoder.transform(marketLabel).toarray()
marketLabel = marketLabel[:,[0]] #1 for absence

#join data back together
marketData = np.column_stack((scalledMarketData, categoricalMarketData))

print(marketData[100])
print(marketLabel[0])
print(marketLabel[1])
print(marketLabel[2])
print(marketData.size)

print(marketLabel.size)

64960
[ 0.38936061  0.85083558 -1.21594825  1.03255877  0.89413719 -0.86156477
 -0.66144918 -0.6378925  -0.61383047 -0.53458349 -0.47975256  0.34941394
 -0.39049128 -0.91089849 -0.8584551   1.51827095 -0.28014028 -0.28383019
 -0.28014028 -0.26211122 -0.11651035 -0.09728167  0.          0.
  2.38884634  0.          0.          1.          0.          0.
  0.          0.          0.          0.          0.          1.
  0.          0.        ]
[1.]
[0.]
[1.]
85120
2240


# Dividing the data into training and testing

In [8]:
Levels = [0,0.1,0.2,0.5,0.9,0.95]

#Divide data, 80% for training, 20% for testing

#Heart data (80% of 303 is 242)
print(heartData.shape)
print(heartLabel.shape)

H_training_data=heartData[0:242,0:13]
H_training_label=heartLabel[0:242]

H_testing_data=heartData[242:303,0:13]
H_testing_label=heartLabel[242:303]

#Marketing data (80% of 2240 is 1792)
print(marketData.shape)
print(marketLabel.shape)

M_training_data=marketData[0:1792,0:38]
M_training_label=marketLabel[0:1792]

M_testing_data=marketData[1792:2240,0:38]
M_testing_label=marketLabel[1792:2240]

#Online Shoppers Data (80% of 12330 is 9864)
print(shopData.shape)
print(shopLabel.shape)

S_training_data=shopData[0:9864,0:33]
S_training_label=shopLabel[0:9864]

S_testing_data=shopData[9864:12330,0:33]
S_testing_label=shopLabel[9864:12330]

(303, 13)
(303, 1)
(2240, 38)
(2240, 1)
(12330, 29)
(12330, 1)


# Self-training algorithm

Heart Data

In [9]:
STrand = np.random.RandomState()

#for calculating average
H_SDT_S = 0
H_RF_S = 0
H_KNN_S = 0
H_MLP_S = 0
H_GB_S = 0

for l in Levels:
  H_SDT = tree.DecisionTreeClassifier()
  H_RF = RandomForestClassifier()
  H_KNN = KNeighborsClassifier()
  H_MLP = MLPClassifier(random_state=1, max_iter=300)
  H_GB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
  H_SVM = SVC() # does not work

  H_random_unlabeled_points = STrand.rand(H_training_label.shape[0]) < l
  H_labels = np.copy(H_training_label)
  H_labels[H_random_unlabeled_points] = -1

  print("Scores when we have ",(l*100),"% of unlabelled data:\n")
  #Single Decision Tree
  H_ST_SDT = SelfTrainingClassifier(H_SDT)
  H_ST_SDT.fit(H_training_data,H_labels.ravel())

  H_ST_SDT_Pred=H_ST_SDT.predict(H_testing_data)
  H_ST_SDT_score = accuracy_score(H_testing_label, H_ST_SDT_Pred)
  H_SDT_S = H_SDT_S + H_ST_SDT_score
  print("Heart SDT Accuracy Score: ",H_ST_SDT_score)

  #Random Forest
  H_ST_RF = SelfTrainingClassifier(H_RF)
  H_ST_RF.fit(H_training_data,H_labels.ravel())

  H_ST_RF_Pred=H_ST_RF.predict(H_testing_data)
  H_ST_RF_score = accuracy_score(H_testing_label, H_ST_RF_Pred)
  H_RF_S = H_RF_S + H_ST_RF_score
  print("Heart RF Accuracy Score: ",H_ST_RF_score)

  #K-nearest neighbor
  H_ST_KNN = SelfTrainingClassifier(H_KNN)
  H_ST_KNN.fit(H_training_data,H_labels.ravel())

  H_ST_KNN_Pred=H_ST_KNN.predict(H_testing_data)
  H_ST_KNN_score = accuracy_score(H_testing_label, H_ST_KNN_Pred)
  H_KNN_S = H_KNN_S + H_ST_KNN_score
  print("Heart KNN Accuracy Score: ",H_ST_KNN_score)

  #Multi-layer Perceptron

  H_ST_MLP = SelfTrainingClassifier(H_MLP)
  H_ST_MLP.fit(H_training_data,H_labels.ravel())

  H_ST_MLP_Pred=H_ST_MLP.predict(H_testing_data)
  H_ST_MLP_score = accuracy_score(H_testing_label, H_ST_MLP_Pred)
  H_MLP_S = H_MLP_S + H_ST_MLP_score
  print("Heart MLP Accuracy Score: ",H_ST_MLP_score)

  #Gradient Boosting
  H_ST_GB = SelfTrainingClassifier(H_GB)
  H_ST_GB.fit(H_training_data,H_labels.ravel())

  H_ST_GB_Pred=H_ST_GB.predict(H_testing_data)
  H_ST_GB_score = accuracy_score(H_testing_label, H_ST_GB_Pred)
  H_GB_S = H_GB_S + H_ST_GB_score
  print("Heart GB Accuracy Score: ",H_ST_GB_score)

  print("\n\n")

  # Support Vector Machine --> base_estimator (Pipeline) should implement predict_proba!
  # H_ST_SVM = SelfTrainingClassifier(H_SVM)
  # H_ST_SVM.fit(H_training_data,H_labels.ravel())

  # H_ST_SVM_Pred=H_ST_SVM.predict(H_testing_data)
  # H_ST_SVM_score = accuracy_score(H_testing_label, H_ST_SVM_Pred)
  # print("Heart SVM Accuracy Score: ",H_ST_SVM_score)

print("Average Heart SDT Accuracy Score: ",H_SDT_S/len(Levels))
print("Average Heart RF Accuracy Score: ",H_RF_S/len(Levels))
print("Average Heart KNN Accuracy Score: ",H_KNN_S/len(Levels))
print("Average Heart MLP Accuracy Score: ",H_MLP_S/len(Levels))
print("Average Heart GB Accuracy Score: ",H_GB_S/len(Levels))

Scores when we have  0 % of unlabelled data:

Heart SDT Accuracy Score:  0.8360655737704918
Heart RF Accuracy Score:  0.8360655737704918
Heart KNN Accuracy Score:  0.6721311475409836


/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


Heart MLP Accuracy Score:  0.819672131147541
Heart GB Accuracy Score:  0.7868852459016393



Scores when we have  10.0 % of unlabelled data:

Heart SDT Accuracy Score:  0.7540983606557377


/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


Heart RF Accuracy Score:  0.8032786885245902
Heart KNN Accuracy Score:  0.639344262295082
Heart MLP Accuracy Score:  0.819672131147541
Heart GB Accuracy Score:  0.819672131147541



Scores when we have  20.0 % of unlabelled data:

Heart SDT Accuracy Score:  0.7868852459016393
Heart RF Accuracy Score:  0.8360655737704918
Heart KNN Accuracy Score:  0.6885245901639344
Heart MLP Accuracy Score:  0.8360655737704918
Heart GB Accuracy Score:  0.7868852459016393



Scores when we have  50.0 % of unlabelled data:

Heart SDT Accuracy Score:  0.7049180327868853
Heart RF Accuracy Score:  0.8524590163934426
Heart KNN Accuracy Score:  0.6557377049180327


...

Heart MLP Accuracy Score:  0.7540983606557377
Heart GB Accuracy Score:  0.8032786885245902



Scores when we have  90.0 % of unlabelled data:

Heart SDT Accuracy Score:  0.6065573770491803
Heart RF Accuracy Score:  0.639344262295082
Heart KNN Accuracy Score:  0.5245901639344263


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Heart MLP Accuracy Score:  0.7049180327868853
Heart GB Accuracy Score:  0.7377049180327869



Scores when we have  95.0 % of unlabelled data:

Heart SDT Accuracy Score:  0.6065573770491803
Heart RF Accuracy Score:  0.7540983606557377
Heart KNN Accuracy Score:  0.6229508196721312


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Heart MLP Accuracy Score:  0.6557377049180327
Heart GB Accuracy Score:  0.7540983606557377



Average Heart SDT Accuracy Score:  0.7158469945355191
Average Heart RF Accuracy Score:  0.7868852459016393
Average Heart KNN Accuracy Score:  0.633879781420765
Average Heart MLP Accuracy Score:  0.7650273224043717
Average Heart GB Accuracy Score:  0.7814207650273225


Marketing Data

In [10]:
STrand = np.random.RandomState()

#for calculating average
M_SDT_S = 0
M_RF_S = 0
M_KNN_S = 0
M_MLP_S = 0
M_GB_S = 0

for l in Levels:
  M_SDT = tree.DecisionTreeClassifier()
  M_RF = RandomForestClassifier()
  M_KNN = KNeighborsClassifier()
  M_MLP = MLPClassifier(random_state=1, max_iter=300)
  M_GB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
  M_SVM = SVC() # does not work

  M_random_unlabeled_points = STrand.rand(M_training_label.shape[0]) < l
  M_labels = np.copy(M_training_label)
  M_labels[M_random_unlabeled_points] = -1

  print("Scores when we have ",(l*100),"% of unlabelled data:\n")
  #Single Decision Tree
  M_ST_SDT = SelfTrainingClassifier(M_SDT)
  M_ST_SDT.fit(M_training_data,M_labels.ravel())

  M_ST_SDT_Pred=M_ST_SDT.predict(M_testing_data)
  M_ST_SDT_score = accuracy_score(M_testing_label, M_ST_SDT_Pred)
  M_SDT_S = M_SDT_S + M_ST_SDT_score
  print("Market SDT Accuracy Score: ",M_ST_SDT_score)

  #Random Forest
  M_ST_RF = SelfTrainingClassifier(M_RF)
  M_ST_RF.fit(M_training_data,M_labels.ravel())

  M_ST_RF_Pred=M_ST_RF.predict(M_testing_data)
  M_ST_RF_score = accuracy_score(M_testing_label, M_ST_RF_Pred)
  M_RF_S = M_RF_S + M_ST_RF_score
  print("Market RF Accuracy Score: ",M_ST_RF_score)

  #K-nearest neighbor
  M_ST_KNN = SelfTrainingClassifier(M_KNN)
  M_ST_KNN.fit(M_training_data,M_labels.ravel())

  M_ST_KNN_Pred=M_ST_KNN.predict(M_testing_data)
  M_ST_KNN_score = accuracy_score(M_testing_label, M_ST_KNN_Pred)
  M_KNN_S = M_KNN_S + M_ST_KNN_score
  print("Market KNN Accuracy Score: ",M_ST_KNN_score)

  #Multi-layer Perceptron

  M_ST_MLP = SelfTrainingClassifier(M_MLP)
  M_ST_MLP.fit(M_training_data,M_labels.ravel())

  M_ST_MLP_Pred=M_ST_MLP.predict(M_testing_data)
  M_ST_MLP_score = accuracy_score(M_testing_label, M_ST_MLP_Pred)
  M_MLP_S = M_MLP_S + M_ST_MLP_score
  print("Market MLP Accuracy Score: ",M_ST_MLP_score)

  #Gradient Boosting
  M_ST_GB = SelfTrainingClassifier(M_GB)
  M_ST_GB.fit(M_training_data,M_labels.ravel())

  M_ST_GB_Pred=M_ST_GB.predict(M_testing_data)
  M_ST_GB_score = accuracy_score(M_testing_label, M_ST_GB_Pred)
  M_GB_S = M_GB_S + M_ST_GB_score
  print("Market GB Accuracy Score: ",M_ST_GB_score)

  print("\n\n")

  # Support Vector Machine --> base_estimator (Pipeline) should implement predict_proba!
  # M_ST_SVM = SelfTrainingClassifier(M_SVM)
  # M_ST_SVM.fit(M_training_data,M_labels.ravel())

  # M_ST_SVM_Pred=M_ST_SVM.predict(M_testing_data)
  # M_ST_SVM_score = accuracy_score(M_testing_label, M_ST_SVM_Pred)
  # print("Market SVM Accuracy Score: ",M_ST_SVM_score)

print("Average Market SDT Accuracy Score: ",M_SDT_S/len(Levels))
print("Average Market RF Accuracy Score: ",M_RF_S/len(Levels))
print("Average Market KNN Accuracy Score: ",M_KNN_S/len(Levels))
print("Average Market MLP Accuracy Score: ",M_MLP_S/len(Levels))
print("Average Market GB Accuracy Score: ",M_GB_S/len(Levels))

Scores when we have  0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


ValueError: ignored

Online Shoppers Data

In [11]:
STrand = np.random.RandomState()

#for calculating average
S_SDT_S = 0
S_RF_S = 0
S_KNN_S = 0
S_MLP_S = 0
S_GB_S = 0

for l in Levels:
  S_SDT = tree.DecisionTreeClassifier()
  S_RF = RandomForestClassifier()
  S_KNN = KNeighborsClassifier()
  S_MLP = MLPClassifier(random_state=1, max_iter=100)
  S_GB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
  S_SVM = SVC() # does not work

  S_random_unlabeled_points = STrand.rand(S_training_label.shape[0]) < l
  S_labels = np.copy(S_training_label)
  S_labels[S_random_unlabeled_points] = -1

  print("Scores when we have ",(l*100),"% of unlabelled data:\n")
  #Single Decision Tree
  S_ST_SDT = SelfTrainingClassifier(S_SDT)
  S_ST_SDT.fit(S_training_data,S_labels.ravel())

  S_ST_SDT_Pred=S_ST_SDT.predict(S_testing_data)
  S_ST_SDT_score = accuracy_score(S_testing_label, S_ST_SDT_Pred)
  S_SDT_S = S_SDT_S + S_ST_SDT_score
  print("Online Shoppers SDT Accuracy Score: ",S_ST_SDT_score)

  #Random Forest
  S_ST_RF = SelfTrainingClassifier(S_RF)
  S_ST_RF.fit(S_training_data,S_labels.ravel())

  S_ST_RF_Pred=S_ST_RF.predict(S_testing_data)
  S_ST_RF_score = accuracy_score(S_testing_label, S_ST_RF_Pred)
  S_RF_S = S_RF_S + S_ST_RF_score
  print("Online Shoppers RF Accuracy Score: ",S_ST_RF_score)

  #K-nearest neighbor
  S_ST_KNN = SelfTrainingClassifier(S_KNN)
  S_ST_KNN.fit(S_training_data,S_labels.ravel())

  S_ST_KNN_Pred=S_ST_KNN.predict(S_testing_data)
  S_ST_KNN_score = accuracy_score(S_testing_label, S_ST_KNN_Pred)
  S_KNN_S = S_KNN_S + S_ST_KNN_score
  print("Online Shoppers KNN Accuracy Score: ",S_ST_KNN_score)

  #Multi-layer Perceptron

  S_ST_MLP = SelfTrainingClassifier(S_MLP)
  S_ST_MLP.fit(S_training_data,S_labels.ravel())

  S_ST_MLP_Pred=S_ST_MLP.predict(S_testing_data)
  S_ST_MLP_score = accuracy_score(S_testing_label, S_ST_MLP_Pred)
  S_MLP_S = S_MLP_S + S_ST_MLP_score
  print("Online Shoppers MLP Accuracy Score: ",S_ST_MLP_score)

  #Gradient Boosting
  S_ST_GB = SelfTrainingClassifier(S_GB)
  S_ST_GB.fit(S_training_data,S_labels.ravel())

  S_ST_GB_Pred=S_ST_GB.predict(S_testing_data)
  S_ST_GB_score = accuracy_score(S_testing_label, S_ST_GB_Pred)
  S_GB_S = S_GB_S + S_ST_GB_score
  print("Online Shoppers GB Accuracy Score: ",S_ST_GB_score)

  print("\n\n")

  # Support Vector Machine --> base_estimator (Pipeline) should implement predict_proba!
  # S_ST_SVM = SelfTrainingClassifier(S_SVM)
  # S_ST_SVM.fit(S_training_data,S_labels.ravel())

  # S_ST_SVS_Pred=S_ST_SVM.predict(S_testing_data)
  # S_ST_SVS_score = accuracy_score(S_testing_label, S_ST_SVS_Pred)
  # print("Online Shoppers SVM Accuracy Score: ",S_ST_SVS_score)

print("Average Online Shoppers SDT Accuracy Score: ",S_SDT_S/len(Levels))
print("Average Online Shoppers RF Accuracy Score: ",S_RF_S/len(Levels))
print("Average Online Shoppers KNN Accuracy Score: ",S_KNN_S/len(Levels))
print("Average Online Shoppers MLP Accuracy Score: ",S_MLP_S/len(Levels))
print("Average Online Shoppers GB Accuracy Score: ",S_GB_S/len(Levels))

Scores when we have  0 % of unlabelled data:

Heart SDT Accuracy Score:  0.8519870235198702


/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


Heart RF Accuracy Score:  0.9014598540145985


/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


Heart KNN Accuracy Score:  0.8673965936739659


...

Heart MLP Accuracy Score:  0.897404703974047
Heart GB Accuracy Score:  0.8957826439578265



Scores when we have  90.0 % of unlabelled data:

Heart SDT Accuracy Score:  0.8491484184914841
Heart RF Accuracy Score:  0.8767234387672344
Heart KNN Accuracy Score:  0.8532035685320357


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multi

Heart MLP Accuracy Score:  0.8860502838605029
Heart GB Accuracy Score:  0.8896999188969992



Scores when we have  95.0 % of unlabelled data:

Heart SDT Accuracy Score:  0.8313057583130575
Heart RF Accuracy Score:  0.8673965936739659
Heart KNN Accuracy Score:  0.8491484184914841


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multi

Heart MLP Accuracy Score:  0.8730738037307381
Heart GB Accuracy Score:  0.8848337388483374



Average Heart SDT Accuracy Score:  0.8524601243579345
Average Heart RF Accuracy Score:  0.8904433630711003
Average Heart KNN Accuracy Score:  0.8620573127872397
Average Heart MLP Accuracy Score:  0.8908488780751554
Average Heart GB Accuracy Score:  0.8921330089213301


# Label Propagation

Heart Data

In [13]:
LPrand = np.random.RandomState()

H_LP_S=0

for l in Levels:
  H_LP = LabelPropagation()

  H_random_unlabeled_pointsLP = LPrand.rand(H_training_label.shape[0]) < l
  H_labelsLP = np.copy(H_training_label)
  H_labelsLP[H_random_unlabeled_pointsLP] = -1

  print("Scores when we have ",(l*100),"% of unlabelled data:\n")

  H_LP.fit(H_training_data, H_labelsLP.ravel())

  H_LP_Pred=H_LP.predict(H_testing_data)
  H_LP_score = accuracy_score(H_testing_label, H_LP_Pred)
  H_LP_S = H_LP_S + H_LP_score
  print("Heart LP Accuracy Score: ",H_LP_score)
  print("\n\n")

print("Average Heart LP Accuracy Score: ",H_LP_S/len(Levels))

Scores when we have  0 % of unlabelled data:

Heart LP Accuracy Score:  0.4918032786885246



Scores when we have  10.0 % of unlabelled data:

Heart LP Accuracy Score:  0.4918032786885246



Scores when we have  20.0 % of unlabelled data:

Heart LP Accuracy Score:  0.5081967213114754



Scores when we have  50.0 % of unlabelled data:

Heart LP Accuracy Score:  0.4918032786885246



Scores when we have  90.0 % of unlabelled data:

Heart LP Accuracy Score:  0.47540983606557374



Scores when we have  95.0 % of unlabelled data:

Heart LP Accuracy Score:  0.5081967213114754



Average Heart LP Accuracy Score:  0.49453551912568305


/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid v

Marketing data

In [14]:
LPrand = np.random.RandomState()

M_LP_S=0

for l in Levels:
  M_LP = LabelPropagation()

  M_random_unlabeled_pointsLP = LPrand.rand(M_training_label.shape[0]) < l
  M_labelsLP = np.copy(M_training_label)
  M_labelsLP[M_random_unlabeled_pointsLP] = -1

  print("Scores when we have ",(l*100),"% of unlabelled data:\n")

  M_LP.fit(M_training_data, M_labelsLP.ravel())

  M_LP_Pred=M_LP.predict(M_testing_data)
  M_LP_score = accuracy_score(M_testing_label, M_LP_Pred)
  M_LP_S = M_LP_S + M_LP_score
  print("Market LP Accuracy Score: ",M_LP_score)
  print("\n\n")

print("Average Market LP Accuracy Score: ",M_LP_S/len(Levels))

Scores when we have  0 % of unlabelled data:



ValueError: ignored

Online Shoppers data

In [15]:
LPrand = np.random.RandomState()

S_LP_S=0

for l in Levels:
  S_LP = LabelPropagation()

  S_random_unlabeled_pointsLP = LPrand.rand(S_training_label.shape[0]) < l
  S_labelsLP = np.copy(S_training_label)
  S_labelsLP[S_random_unlabeled_pointsLP] = -1

  print("Scores when we have ",(l*100),"% of unlabelled data:\n")

  S_LP.fit(S_training_data, S_labelsLP.ravel())

  S_LP_Pred=S_LP.predict(S_testing_data)
  S_LP_score = accuracy_score(S_testing_label, S_LP_Pred)
  S_LP_S = S_LP_S + S_LP_score
  print("Online Shoppers LP Accuracy Score: ",S_LP_score)
  print("\n\n")

print("Average Online Shoppers LP Accuracy Score: ",S_LP_S/len(Levels))

Scores when we have  0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer


Heart LP Accuracy Score:  0.8345498783454988



Scores when we have  10.0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer


Heart LP Accuracy Score:  0.8337388483373885



Scores when we have  20.0 % of unlabelled data:



...

/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer


Heart LP Accuracy Score:  0.813463098134631



Scores when we have  95.0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:318: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning,


Heart LP Accuracy Score:  0.8017031630170316



Average Heart LP Accuracy Score:  0.8238037307380374


/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:222: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer


# Semi-supervised Ensemble

Heart Data

In [16]:
SSErand = np.random.RandomState()

#for calculating average
H_SSE_S = 0

for l in Levels:
  #most successsful in self-training
  H_RF_SSE = RandomForestClassifier()
  H_ST_SSE = SelfTrainingClassifier(H_RF_SSE)

  H_random_unlabeled_pointsSSE = SSErand.rand(H_training_label.shape[0]) < l
  H_labelsSSE = np.copy(H_training_label)
  H_labelsSSE[H_random_unlabeled_pointsSSE] = -1

  print("Scores when we have ",(l*100),"% of unlabelled data:\n")

  H_SSE = BaggingClassifier(H_ST_SSE,n_estimators=10, random_state=0)
  H_SSE.fit(H_training_data,H_labelsSSE.ravel())

  H_SEE_Pred=H_SSE.predict(H_testing_data)
  H_SSE_score = accuracy_score(H_testing_label, H_SEE_Pred)
  H_SSE_S = H_SSE_S + H_SSE_score
  print("Heart RF Accuracy Score: ",H_SSE_score)
  print("\n\n")

print("Average Heart SSE Accuracy Score: ",H_SSE_S/len(Levels))

#Heart data
# SDT2 = tree.DecisionTreeClassifier()
# self_training_model2 = SelfTrainingClassifier(SDT2)

# label_prop_model2 = LabelPropagation()
# rng3 = np.random.RandomState()
# random_unlabeled_points2 = rng.rand(Htraining_label.shape[0]) < 0.3
# labels3 = np.copy(Htraining_label)
# labels3[random_unlabeled_points2] = -1

# print(labels3.ravel())

# clf = BaggingClassifier(self_training_model2,n_estimators=10, random_state=0)
# clf.fit(Htraining_data, labels3.ravel())

# ensemble_Pred=clf.predict(Htesting_data)
# print(ensemble_Pred)

# score = accuracy_score(Htesting_label, ensemble_Pred)
# print("Accuracy Score: ",score)

#Marketing data

#Online Shoppers data

Scores when we have  0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/

Heart RF Accuracy Score:  0.8360655737704918



Scores when we have  10.0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/

Heart RF Accuracy Score:  0.8524590163934426



Scores when we have  20.0 % of unlabelled data:



...

Heart RF Accuracy Score:  0.3114754098360656



Scores when we have  90.0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/

Heart RF Accuracy Score:  0.0



Scores when we have  95.0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/

Heart RF Accuracy Score:  0.0



Average Heart SSE Accuracy Score:  0.47267759562841527


Market Data

In [17]:
SSErand = np.random.RandomState()

#for calculating average
M_SSE_S = 0

for l in Levels:
  #most successsful in self-training
  M_RF_SSE = RandomForestClassifier()
  M_ST_SSE = SelfTrainingClassifier(M_RF_SSE)

  M_random_unlabeled_pointsSSE = SSErand.rand(M_training_label.shape[0]) < l
  M_labelsSSE = np.copy(M_training_label)
  M_labelsSSE[M_random_unlabeled_pointsSSE] = -1

  print("Scores when we have ",(l*100),"% of unlabelled data:\n")

  M_SSE = BaggingClassifier(M_ST_SSE,n_estimators=10, random_state=0)
  M_SSE.fit(M_training_data,M_labelsSSE.ravel())

  M_SEE_Pred=M_SSE.predict(M_testing_data)
  M_SSE_score = accuracy_score(M_testing_label, M_SEE_Pred)
  M_SSE_S = M_SSE_S + M_SSE_score
  print("Market RF Accuracy Score: ",M_SSE_score)
  print("\n\n")

print("Average Market SSE Accuracy Score: ",M_SSE_S/len(Levels))


Scores when we have  0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


ValueError: ignored

Online Shoppers Data

In [18]:
SSErand = np.random.RandomState()

#for calculating average
S_SSE_S = 0

for l in Levels:
  #most successsful in self-training
  S_RF_SSE = RandomForestClassifier()
  S_ST_SSE = SelfTrainingClassifier(S_RF_SSE)

  S_random_unlabeled_pointsSSE = SSErand.rand(S_training_label.shape[0]) < l
  S_labelsSSE = np.copy(S_training_label)
  S_labelsSSE[S_random_unlabeled_pointsSSE] = -1

  print("Scores when we have ",(l*100),"% of unlabelled data:\n")

  S_SSE = BaggingClassifier(S_ST_SSE,n_estimators=10, random_state=0)
  S_SSE.fit(S_training_data,S_labelsSSE.ravel())

  S_SEE_Pred=S_SSE.predict(S_testing_data)
  S_SSE_score = accuracy_score(S_testing_label, S_SEE_Pred)
  S_SSE_S = S_SSE_S + S_SSE_score
  print("Online Shoppers RF Accuracy Score: ",S_SSE_score)
  print("\n\n")

print("Average Online Shoppers SSE Accuracy Score: ",S_SSE_S/len(Levels))

Scores when we have  0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/

Online Shoppers RF Accuracy Score:  0.9014598540145985



Scores when we have  10.0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/

Online Shoppers RF Accuracy Score:  0.9022708840227088



Scores when we have  20.0 % of unlabelled data:



...

Online Shoppers RF Accuracy Score:  0.33657745336577455



Scores when we have  90.0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/

Online Shoppers RF Accuracy Score:  0.0008110300081103001



Scores when we have  95.0 % of unlabelled data:



/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)
/usr/local/lib/python3.7/

Online Shoppers RF Accuracy Score:  0.0



Average Online Shoppers SSE Accuracy Score:  0.5068937550689375
